## TOF - first light analysis

Check monitoring data and paddle baslines/RMMS as well as 
2d charge distributions

In [ ]:
import gaps_online as go
import gaps_online.db as db
import polars as pl
import numpy as np
import scipy.integrate as integrate
import HErmes as he
import HErmes.fitting as fit
import scipy.stats as st
from scipy.spatial.transform import Rotation as rot

from pathlib import Path
import dashi as d
d.visual()
import tqdm

import matplotlib.pyplot as plt
import charmingbeauty as cb
lo = cb.layout
cb.visual.set_style_present()

import re
!export DJANGO_ALLOW_ASYNC_UNSAFE=1
import os
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = '1'

In [ ]:
C_LIGHT_PADDLE = 15.4; 

def get_ts_from_files(files):
    pattern = re.complie('Run[0-9]*_[0-9]*.(?P<tdate>[0-9_]*)')
    ts = pattern.search(files)[0]
    print (ts)
    
def calc_rms(data):         
    return np.sqrt((data**2).sum()/len(data))

def get_t0(cfd_a, cfd_b, paddle_len):
    return 0.5*(cfd_a + cfd_b - (paddle_len/(10.0*C_LIGHT_PADDLE)))
    
def get_pos(cfd_a, t0):
    return (cfd_a - t0)*C_LIGHT_PADDLE*10.0 

def construct_plen_table():
    paddles = db.Paddle.objects.all()
    plen = dict()
    for pdl in paddles:
        plen[pdl.paddle_id] = pdl.length
    return plen

def get_hit_paddles_rbs(ev, paddles):
    hit_paddles = []
    for rbev in ev.rbevents:
        chnls = np.array(rbev.header.get_channels())
        chnls += 1
        for pdl in paddles:
            if pdl.rb_id == rbev.header.rb_id:
                if pdl.rb_chA in chnls:
                    hit_paddles.append((pdl.paddle_id, pdl.panel_id))
                    continue
    return hit_paddles

def get_hit_paddles(hits, paddles):
    hit_paddles = []
    for h in hits:
        for pdl in paddles:
            if pdl.dsi == h[0]:
                if pdl.j_ltb == h[1]:
                    ch = (pdl.ltb_chA, pdl.ltb_chB)
                    if sorted(ch) == sorted(h[2]):
                        hit_paddles.append((pdl.paddle_id, pdl.panel_id, (pdl.global_pos_x_l0, pdl.global_pos_y_l0, pdl.global_pos_z_l0)))
                        break
    return hit_paddles

def charge_dist(charges, bins):
    """
    """   

    def Landau(xs, scale, mu, eta):
        return scale*st.moyal.pdf(xs, loc=mu, scale=eta )

    fit = he.fitting.model.Model(Landau)

    fig = plt.figure(figsize=cb.layout.FIGSIZE_A4_LANDSCAPE)
    ax = fig.gca()
    h = d.factory.hist1d(charges, bins)

    spectral = h.bincenters, h.bincontent
    fit.startparams = (max(spectral[1]), 1000 ,1111.15)
    fit.add_data(h.bincontent, xs=h.bincenters, create_distribution=False)
    fit.fit_to_data(silent=True)
    h.line(filled=True, alpha=0.7,color='r')
    ax.plot(bins, fit(bins, *fit.best_fit_params), color='r', label='Landau fit')
    ax.set_xlabel('ccharge [pC]', loc='right')
    ax.set_ylabel('entries', loc='top')
    ax.legend()
    return fig

def get_rot(axis, theta):
    if axis == 'x':
        mat = np.array([[1,0,0],
                       [0,np.cos(theta),-np.sin(theta)],
                       [0,np.sin(theta), np.cos(theta)]])

    if axis == 'y':
        mat = np.array([[np.cos(theta),0 ,np.sin(theta)],
                       [0,1,0],
                       [-np.sin(theta), 0, np.cos(theta)]])

    if axis == 'z':
        mat = np.array([[np.cos(theta),-np.sin(theta),0],
                       [np.sin(theta),np.cos(theta),0],
                       [0,0, 1]])
    return mat

#print (get_rot('x', np.pi/2))


In [ ]:
dataset = Path('/data0/gaps/csbf/csbf-data/63/')
dataset = Path('/data0/gaps/csbf/csbf-data/81/')

files = [f for f in sorted(dataset.glob('*.tof.gaps'))]
print(f'-> Got {len(files)} run files!')

In [ ]:
print (files)
#files = [files[1],files[3]]
files = [files[0],files[2]]

In [ ]:
event_ids = []
for f in files:
    reader = go.rust_api.io.TofPacketReader(str(f), filter=go.rust_api.io.PacketType.TofEvent)
    for pack in reader:
        ev = go.rust_api.events.TofEvent()
        ev.from_tofpacket(pack)
        event_ids.append(ev.event_id)


print (len(event_ids))
print (sorted(event_ids)[-1])
print (sorted(event_ids)[0])
409566406 in event_ids

In [ ]:
# monitoring data
rbmoni = go.rust_api.moni.RBMoniSeries()
dfs = []
for f in tqdm.tqdm_notebook(files):
#for f in tqdm.notebook.tqdm(files):
    df = rbmoni.from_file(str(f))
    dfs.append(df)

In [ ]:
df_merged = dfs[0]
for df in dfs[1:]:
    df_merged.vstack(df)

In [ ]:
df_merged.filter(pl.col('board_id') == 1)
boards  = df_merged['board_id']
boards = [int(k) for k in set(boards)]

In [ ]:
df.columns

The following cell will plot all RBMoni data for all boards

In [ ]:
%matplotlib inline

cb.set_style_present()

met_tick = 10

for k in boards:
    for var in df.columns:
        fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE_HALF_HEIGHT)
        ax = fig.gca()
        data = df.filter(pl.col('board_id') == k)[var].to_numpy()
        met = np.arange(0,len(data), 1)*10
        ax.set_title(f'RB {k}', loc='right')
        ax.set_xlabel('MET [s]', loc='right')
        ax.set_ylabel(var,loc='top')
        ax.plot(met,data)
plt.show()

In [ ]:
cali_dir   = Path('/data0/gaps/csbf/csbf-data/calib/240622_200126UTC')
cali_dir   = Path('/data0/gaps/csbf/csbf-data/calib/240702_215856UTC')
print (cali_dir)
cali = go.tof.calibrations.load_calibrations_rapi(cali_dir)

In [ ]:
# check the cxx bindings
cxx_testfile = str(files[0])
cxxs_testfile = '/data0/gaps/csbf/csbf-data/72/Run72_1.240713_022959UTC.tof.gaps'
cxx_packets = go.cxx_api.get_tofpackets(cxx_testfile, filter=go.cxx_api.PacketType.TofEvent)
cxx_ev = go.cxx_api.TofEvent.from_tofpacket(cxx_packets[7000])
print (cxx_ev)
for k in cxx_ev.rbevents:
    print (k)

rust_reader = go.rust_api.io.TofPacketReader(cxx_testfile, filter=go.rust_api.io.PacketType.TofEvent)
rust_packets = [k for k in rust_reader]
rust_events =  []
for pack in rust_packets:
    rev = go.rust_api.events.TofEvent()
    rev.from_tofpacket(pack)
    rust_events.append(rev)
print ('======================')
print (rust_events[7000])
#print (rust_events[7000].rbevents[0].hits)
rbevent = rust_events[7000].rbevents[0]
print(rust_events[7000].hits)
#rbevent.

In [ ]:
# event data
BL_BINS = slice(850,950)
all_paddles = db.Paddle.objects.all()
sides   = {k : dict() for k in range(50)}
for pdl in all_paddles:
#    #print (rb)
    sides[pdl.rb_id].update({ pdl.rb_chA : ('A', pdl.paddle_id), pdl.rb_chB : ('B', pdl.paddle_id)})

paddles_bl_a = { k : [] for k in range(1,161)}
paddles_bl_b = { k : [] for k in range(1,161)}
   
paddles_bl_a_rms  = { k : [] for k in range(1, 161)}
paddles_bl_b_rms  = { k : [] for k in range(1, 161)}

paddles_charge_a  = { k : [] for k in range(1,161)}
paddles_charge_b  = { k : [] for k in range(1,161)}

paddles_peak_a  = { k : [] for k in range(1,161)}
paddles_peak_b  = { k : [] for k in range(1,161)}

n_trigger_hits = []
trigger_panels = []

empty_hits = []
timestamps = []
event_ids  = []
for f in tqdm.tqdm_notebook(files):
    reader = go.rust_api.io.TofPacketReader(str(f), filter=go.rust_api.io.PacketType.TofEvent)
    for pack in reader:
        ev = go.rust_api.events.TofEvent()
        ev.from_tofpacket(pack)
        timestamps.append(ev.mastertriggerevent.timestamp_abs48)
        event_ids.append(ev.mastertriggerevent.event_id)
        trigger_hits = ev.mastertriggerevent.trigger_hits
        if len(trigger_hits) == 0:
            empty_hits.append(ev)
        n_trigger_hits.append(len(trigger_hits))
        trigger_hits = get_hit_paddles(trigger_hits, all_paddles)
        trigger_panels.extend([k[1] for k in trigger_hits])
        for wf in ev.waveforms:
            wf.calibrate(cali[wf.rb_id])
            rms = calc_rms(wf.voltages[BL_BINS])
            mn  = wf.voltages[BL_BINS].mean()
            charge = integrate.simpson(wf.voltages[10:] - mn, x= wf.times[10:])
            peak   = max(wf.voltages[10:])
            #print (f'wf mean {mn}, rms {rms}')
            #waveforms.append(wf)
            if wf.rb_channel == 8:
                continue
            side, pdl_id = sides[wf.rb_id][wf.rb_channel + 1]
            if side == 'A':
                paddles_bl_a[pdl_id].append(mn)
                paddles_bl_a_rms[pdl_id].append(rms)
                paddles_charge_a[pdl_id].append(charge)
                paddles_peak_a[pdl_id].append(peak)
            else:
                paddles_bl_b[pdl_id].append(mn)
                paddles_bl_b_rms[pdl_id].append(rms)
                paddles_charge_b[pdl_id].append(charge)
                paddles_peak_b[pdl_id].append(peak)

In [ ]:
%matplotlib inline
#print (sorted(timestamps)[:100])
#print (sorted(event_ids)[:100])

figa = plt.figure(figsize=cb.layout.FIGSIZE_A4_LANDSCAPE)
ax  = figa.gca()
ax.set_ylabel('TOF timestamp [clock cycles]', loc='top')
ax.set_xlabel('entry')
ax.plot(timestamps)

figb = plt.figure(figsize=cb.layout.FIGSIZE_A4_LANDSCAPE)
ax  = figb.gca()
ax.set_ylabel('TOF Event ID', loc='top')
ax.set_xlabel('entry')
ax.plot(event_ids)

figc = plt.figure(figsize=cb.layout.FIGSIZE_A4_LANDSCAPE)
ax  = figc.gca()
ax.set_title('Run 63')
ax.set_ylabel('TOF timestamp [clock cycles]', loc='top')
ax.set_xlabel('TOF Event ID', loc='right')
ax.scatter(event_ids, timestamps)

plt.show()

In [ ]:
print (len(n_trigger_hits))
fig = plt.figure(figsize=cb.layout.FIGSIZE_A4_LANDSCAPE)
ax  = fig.gca()
h   = d.factory.hist1d(n_trigger_hits, bins=np.arange(-0.5,25.5,1))
h.line(filled=True, alpha=0.8, color='b')
cb.visual.adjust_minor_ticks(ax)
print (h.bincenters, h.bincontent)
ax.set_ylim(bottom=0)
ax.set_xlim(left=0.5)
ax.set_xlabel('$N_{trig.\; hits}$', loc='right')
ax.set_ylabel('entries', loc='top')


fig = plt.figure(figsize=cb.layout.FIGSIZE_A4_LANDSCAPE)
ax  = fig.gca()
h   = d.factory.hist1d(trigger_panels, bins=np.arange(0.5,25.5,1))
h.line(filled=True, alpha=0.8, color='b')
cb.visual.adjust_minor_ticks(ax)
ax.set_ylim(bottom=0)
ax.set_xlim(left=0.5)
ax.set_xlabel('panel ID', loc='right')
ax.set_ylabel('entries', loc='top')

plt.show()

In [ ]:
#print (empty_hits[0].mastertriggerevent)
for j in empty_hits:
    print (get_hit_paddles_rbs(j, all_paddles))
    for k in j.rbevents:
        for wf in k.waveforms:
            wf.calibrate(cali[wf.rb_id])
            if wf.rb_channel == 8:
                continue
            fig = plt.figure(figsize=cb.layout.FIGSIZE_A4_LANDSCAPE_HALF_HEIGHT)
            ax = fig.gca()
            ax.plot(wf.times[10:], wf.voltages[10:], lw=1.2)
            

plt.show()

In [ ]:
ev_hits = [k for k in  get_hit_paddles(ev.mastertriggerevent.get_trigger_hits(), all_paddles)]
print(ev_hits)
ev_hits = [k[2] for k in  get_hit_paddles(ev.mastertriggerevent.get_trigger_hits(), all_paddles)]
print (ev_hits)

In [ ]:
%matplotlib widget

from mpl_toolkits.mplot3d.art3d import Poly3DCollection

def plot_panel(ax,panel, colors):
    print (panel.normal_x, panel.normal_y, panel.normal_z)
    panel_normal = np.array(panel.normal_x, panel.normal_y, panel.normal_z])
    for paddle in panel.paddles:
        center = np.array([paddle.global_pos_x_l0,\
                           paddle.global_pos_y_l0,\
                           paddle.global_pos_z_l0])
        direction = np.array([paddle.length, paddle.width, paddle.height]   
        if panel.normal
        if panel.panel_id in [2,7]:
            y = paddle.global_pos_x_l0
            x = paddle.global_pos_y_l0 - paddle.length/2
            z = paddle.global_pos_z_l0
            l = paddle.length
            w = paddle.width
            h = paddle.height
            #print (dir(paddle))
        if panel.panel_id == 1:
            l = paddle.length
            w = paddle.width
            h = paddle.height
            y = paddle.global_pos_x_l0
            x = paddle.global_pos_y_l0 - paddle.length/2
            z = paddle.global_pos_z_l0
        if panel.panel_id in [3]:
            l = paddle.length
            h = paddle.width
            w = paddle.height
            y = paddle.global_pos_x_l0
            x = paddle.global_pos_y_l0 - paddle.length/2
            z = paddle.global_pos_z_l0
        if panel.panel_id in [4]:
            w = paddle.length
            h = paddle.width
            l = paddle.height
            x = paddle.global_pos_x_l0 + paddle.length/2
            y = paddle.global_pos_y_l0 - paddle.length
            z = paddle.global_pos_z_l0
        vertices = np.array([\
            [x,y,z],
            [x+l,y,z],
            [x+l,y+w,z],
            [x,y+w,z],
            [x,y,z+h],
            [x+l,y,z+h],
            [x+l,y+w,z+h],
            [x,y+w,z+h]])
    
        faces = [\
            [vertices[0], vertices[1], vertices[2], vertices[3]],
            [vertices[4], vertices[5], vertices[6], vertices[7]],
            [vertices[0], vertices[1], vertices[5], vertices[4]],
            [vertices[1], vertices[2], vertices[6], vertices[5]],
            [vertices[2], vertices[3], vertices[7], vertices[6]],
            [vertices[3], vertices[0], vertices[4], vertices[7]]]
        ax.add_collection3d(Poly3DCollection(faces, facecolors=colors,edgecolors='gray', alpha=0.3))

paddle = all_paddles[0]
panels  = db.Panel.objects.all()

vertices = np.array([\
    [0,0,0],
    [1,0,0],
    [1,1,0],
    [0,1,0],
    [0,0,1],
    [1,0,1],
    [1,1,1],
    [0,1,1]])



# set up the figure and Axes
fig = plt.figure(figsize=(8, 3))
ax1 = fig.add_subplot(121, projection='3d')
xs = [k[0] for k in ev_hits]
ys = [k[1] for k in ev_hits]
zs = [k[2] for k in ev_hits]
ax1.scatter(xs, ys, zs, color='k')
panels = [panels[0],panels[1],
          panels[2],panels[3],
          #panels[4],#panels[5],
          panels[6]]
for panel in panels:
    plot_panel(ax1,panel,'b')
#ax1.add_collection3d(poly, zs=zs)
#for paddle in all_paddles[:12]:
#    plot_paddle(ax1, paddle, 'b')
ax1.set_aspect('equal')

In [ ]:
print(

In [ ]:
header = ev.rbevents[0].header
header.get_channels()

In [ ]:
paddles = db.Paddle.objects.all()
#print (paddles[0])
_pdl = paddles[0]
testhits = ev.mastertriggerevent.get_trigger_hits()
for h in testhits:
    print(f'--{h}')
print(get_hit_paddles(ev.mastertriggerevent.get_trigger_hits(), paddles))
for h in ev.hits:
    print (h)

In [ ]:
%matplotlib inline

bl_bins = np.linspace(-2,2,120)

for k in range(1,161):   
    fig = p.figure(figsize=lo.FIGSIZE_A4_SQUARE)
    ax  = fig.gca()
    for i,data in enumerate([paddles_bl_a[k], paddles_bl_b[k]]):
        if i == 0:
            side   = 'A'
            color  = 'b'
            ycoord = 0.85
        else:
            side   = 'B'
            color  = 'r'
            ycoord = 0.65
        #print(len(paddles_bl_a[k]))
        data = np.array(data)

        h = d.factory.hist1d(data, bl_bins)
        h.line(filled=True, alpha=0.6, color=color)
        ax.set_xlabel('baseline [mV]', loc='right')
        ax.set_ylabel('entries', loc='top')
        ax.text(0.05,ycoord, \
                f'Paddle {k}{side}\nN = {len(data)}\n $\\mu$ = {data.mean():.2}\n$\\sigma$ = {data.std():.2}',\
                transform=ax.transAxes,\
                color=color
        )
        ax.set_ylim(bottom=0)
        cb.visual.adjust_minor_ticks(ax)
    #break

p.show()

In [ ]:
%matplotlib inline

bl_bins = np.linspace(0,2,120)

for k in range(1,161):   
    fig = p.figure(figsize=lo.FIGSIZE_A4_SQUARE)
    ax  = fig.gca()
    for i,data in enumerate([paddles_bl_a_rms[k], paddles_bl_b_rms[k]]):
        if i == 0:
            side   = 'A'
            color  = 'b'
            ycoord = 0.85
        else:
            side   = 'B'
            color  = 'r'
            ycoord = 0.65
        #print(len(paddles_bl_a[k]))
        data = np.array(data)

        h = d.factory.hist1d(data, bl_bins)
        h.line(filled=True, alpha=0.6, color=color)
        ax.set_xlabel('baseline RMS [mV]', loc='right')
        ax.set_ylabel('entries', loc='top')
        ax.text(0.05,ycoord, \
                f'Paddle {k}{side}\nN = {len(data)}\n $\\mu$ = {data.mean():.2}\n$\\sigma$ = {data.std():.2}',\
                transform=ax.transAxes,\
                color=color
        )
        ax.set_ylim(bottom=0)
        cb.visual.adjust_minor_ticks(ax)
    #break

p.show()

In [ ]:
%matplotlib inline


for k in range(1,161):   
    fig = p.figure(figsize=lo.FIGSIZE_A4_SQUARE)
    ax  = fig.gca()
    ch_a = paddles_charge_a[k]
    ch_b = paddles_charge_b[k]

    #print(len(paddles_bl_a[k]))
    # = np.array(data)

    #h = d.factory.hist1d(data, bl_bins)
    #h.line(filled=True, alpha=0.6, color=color)
    ax.set_xlabel('charge A [pC]', loc='right')
    ax.set_ylabel('charge B [pC]', loc='top')
    ax.scatter(ch_a, ch_b, s=1, marker='+')
    ax.text(0.05,ycoord, \
     f'Paddle {k}',\
     transform=ax.transAxes,\
    #color=color
    )
    ax.set_ylim(bottom=0)
    cb.visual.adjust_minor_ticks(ax)
    #break

p.show()

In [ ]:
waveforms = []
for f in tqdm.notebook.tqdm(files):
    reader = go.rust_api.io.TofPacketReader(str(f), filter=go.rust_api.io.PacketType.TofEvent)
    for pack in reader:
        ev = go.rust_api.events.TofEvent()
        ev.from_tofpacket(pack)
        for wf in ev.waveforms:
            if wf.rb_channel == 8:
                continue
            side, pdl_id = sides[wf.rb_id][wf.rb_channel + 1]
            if pdl_id != 1:
                continue
            wf.calibrate(cali[wf.rb_id])
            waveforms.append(wf)

In [ ]:
charges_a     = []
charges_b     = []
wf_a          = []
wf_b          = []
#last_evid = waveforms[0].event_id
for wf in tqdm.notebook.tqdm_notebook(waveforms):
    #print(wf)
    bl     = wf.voltages[BL_BINS].mean()
    charge = integrate.simpson(wf.voltages[10:] -bl, x=wf.times[10:])
    side, pdl_id = sides[wf.rb_id][wf.rb_channel + 1]
    #if wf.event_id != last_evid:
    #    last_evid = wf.event_id
    #    continue
    #last_evid = wf.event_id
    if side == 'A':
        charges_a.append(charge)
        wf_a.append(wf)
    else:
        charges_b.append(charge)
        wf_b.append(wf)
    


In [ ]:
print (len(charges_a), len(charges_b), len(wf_a))

In [ ]:
charges_a = np.array(charges_a)
charges_b = np.array(charges_b)

ratio     = charges_a/charges_b
bins      = np.linspace(0,10, 150)

fig = plt.figure(figsize=lo.FIGSIZE_A4_SQUARE)
ax  = fig.gca()
h   = d.factory.hist1d(ratio, bins)
h.line(filled=True, alpha=0.5, color='b')
cb.visual.adjust_minor_ticks(ax)
ax.set_ylim(bottom=0)

fig = plt.figure(figsize=lo.FIGSIZE_A4_SQUARE)
ax = fig.gca()
ax.set_xlabel('charge A [pC]', loc='right')
ax.set_ylabel('charge B [pC]', loc='top')
ax.scatter(ch_a, ch_b, s=1, marker='+')
cb.visual.adjust_minor_ticks(ax, which='both')

fig = plt.figure(figsize=lo.FIGSIZE_A4_SQUARE)
ax = fig.gca()
ax.set_xlabel('charge [pC]', loc='right')
ax.set_ylabel('entries', loc='top')
h_a = d.factory.hist1d(ch_a, bins=np.linspace(0,50000, 150))
h_b = d.factory.hist1d(ch_b, bins=np.linspace(0,50000, 150))
h_a.line(filled=True, alpha=0.7, color='b')
h_b.line(filled=True, alpha=0.7, color='r')
ax.set_yscale('log')
cb.visual.adjust_minor_ticks(ax, which='both')

charge_dist(ch_a, np.linspace(0,10000,100))

p.show()

In [ ]:
# outliers
charges_a_cut = []
charges_b_cut = []
wf_a_cut      = []
wf_b_cut      = []
for k in range(len(charges_a)):
    if charges_b[k] > 5000 and charges_a[k] < 5000:
        charges_a_cut.append(charges_a[k])
        charges_b_cut.append(charges_b[k])
        wf_a_cut.append(wf_a[k])
        wf_b_cut.append(wf_b[k])

print (len(charges_a_cut))

In [ ]:
charges_a_cut = np.array(charges_a_cut)
charges_b_cut = np.array(charges_b_cut)

ratio     = charges_a_cut/charges_b_cut
bins      = np.linspace(0,10, 150)

fig = plt.figure(figsize=lo.FIGSIZE_A4_SQUARE)
ax  = fig.gca()
h   = d.factory.hist1d(ratio, bins)
h.line(filled=True, alpha=0.5, color='b')

fig = plt.figure(figsize=lo.FIGSIZE_A4_SQUARE)
ax = fig.gca()
ax.set_xlabel('charge A [pC]', loc='right')
ax.set_ylabel('charge B [pC]', loc='top')
ax.scatter(charges_a_cut, charges_b_cut, s=1, marker='+')
p.show()

In [ ]:
plengths = construct_plen_table()

plot = True
#for k in range(len(charges_a_cut)):
for k in range(10):
    charge = charges_a_cut[k]
    peaks = go.rust_api.analysis.find_peaks(wf_a_cut[k].voltages[10:], wf_a_cut[k].times[10:], 50, 300,3, 10, 2)
    cfds = []
    for pk in peaks:
        ax.axvspan(wf_a_cut[k].times[pk[0]],\
                   wf_a_cut[k].times[pk[1]], color='b', alpha=0.2)
        cfd = go.rust_api.analysis.cfd_simple(wf_a_cut[k].voltages[10:],\
                                              wf_a_cut[k].times[10:],\
                                              0.2,
                                              pk[0],
                                              pk[1])
        cfds.append(cfd)


    cfd_a = cfds[0]
    if plot:
        fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE_HALF_HEIGHT)
        ax  = fig.gca()
        ax.plot(wf_a_cut[k].times[10:], wf_a_cut[k].voltages[10:], color='b', lw=1.2, alpha=0.9)
        ax.scatter(cfds, [0 for j in range(len(cfds))], marker='v', color='b', s=40)
        ax.text(0.55,0.8, \
         f'A: {charge:.2f} pC, t: {cfd_a:.2f}',\
         transform=ax.transAxes,\
         color='b'
        )

    # b-side
    peaks = go.rust_api.analysis.find_peaks(wf_b_cut[k].voltages[10:], wf_b_cut[k].times[10:], 50, 300,3, 10, 2)
    cfds = []
    for pk in peaks:
        ax.axvspan(wf_a_cut[k].times[pk[0]],\
                   wf_a_cut[k].times[pk[1]], color='r', alpha=0.2)
        cfd = go.rust_api.analysis.cfd_simple(wf_b_cut[k].voltages[10:],\
                                              wf_b_cut[k].times[10:],\
                                              0.2,
                                              pk[0],
                                              pk[1])
        cfds.append(cfd)

    cfd_b = cfds[0]
    t0 = get_t0(cfd_a, cfd_b, plengths[1]*10)
    ps = get_pos(cfd_a, t0)
    if plot:
        ax.plot(wf_b_cut[k].times[10:], wf_b_cut[k].voltages[10:], color='r', lw=1.2, alpha=0.9)
        ax.scatter(cfds, [0 for j in range(len(cfds))], marker='v', color='r', s=40)
        charge = charges_b_cut[k]
        ax.text(0.55,0.65, \
         f'B: {charge:.2f} pC, t: {cfd_b:.2f}',\
         transform=ax.transAxes,\
         color='r'
        )
        delta_t = cfd_a - cfd_b
        ax.text(0.55,0.3, \
         f'{ps:.2f} [mm], $t_0$ : {t0:.2f}\n $\Delta$t : {delta_t:.2f}',\
         transform=ax.transAxes,\
         color='k'
        )

        cb.visual.adjust_minor_ticks(ax, which='both')
        ax.set_xlabel('ns', loc='right')
        ax.set_ylabel('mV', loc='top')
p.show()

In [ ]:
t0s        = []
positions  = []
delta_tdcs = []

for k in tqdm.notebook.tqdm(range(len(wf_a))):
    peaks = go.rust_api.analysis.find_peaks(wf_a[k].voltages[10:], wf_a[k].times[10:], 50, 300,3, 10, 2)
    cfds = []
    for pk in peaks:
        cfd = go.rust_api.analysis.cfd_simple(wf_a[k].voltages[10:],\
                                              wf_a[k].times[10:],\
                                              0.2,
                                              pk[0],
                                              pk[1])
        cfds.append(cfd)
        break


  

    # b-side
    peaks = go.rust_api.analysis.find_peaks(wf_b[k].voltages[10:], wf_b[k].times[10:], 50, 300,3, 10, 2)
    for pk in peaks:
        cfd = go.rust_api.analysis.cfd_simple(wf_b[k].voltages[10:],\
                                              wf_b[k].times[10:],\
                                              0.2,
                                              pk[0],
                                              pk[1])
        cfds.append(cfd)
        break

    if len(cfds) != 2:
        continue
    cfd_a = cfds[0]
    cfd_b = cfds[1]
    t0 = get_t0(cfd_a, cfd_b, plengths[1]*10)
    ps = get_pos(cfd_a, t0)
    delta_tdc = cfd_b - cfd_a
    t0s.append(t0)
    positions.append(ps)
    delta_tdcs.append(delta_tdc)

In [ ]:
fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE_HALF_HEIGHT)
ax  = fig.gca()
h   = d.factory.hist1d(delta_tdcs, np.linspace(-15,15,100))
h.line(color='b', filled=True, alpha=0.7)
ax.set_ylim(bottom=0)
p.show()

In [ ]:
fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE_HALF_HEIGHT)
ax  = fig.gca()

h   = d.factory.hist1d(positions, np.linspace(0,2000,100))
h.line(color='b', filled=True, alpha=0.7)
ax.set_ylim(bottom=0)
p.show()

In [ ]:
fig = plt.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE_HALF_HEIGHT)
ax  = fig.gca()

h   = d.factory.hist1d(t0s, np.linspace(0,200,100))
h.line(color='b', filled=True, alpha=0.7)
ax.set_ylim(bottom=0)
p.show()

In [ ]:
rbcali = go.rust_api.events.RBCalibration()
rbcali.from_file('/data0/gaps/nevis/calib/latest/RB01_2024_03_28-17_26_44.cali.tof.gaps')
rbcali

In [ ]:
print (len(waveforms))

In [ ]:
go.rust_api.analysis.find_peaks?

In [ ]:
%%time

import os.path

avg_wf = np.zeros(1024)

npacks = 0
nwfs   = 0
read_bytes = 0
finish = False
for rf in run_files[10:30]:
    print (f'-> Reading {rf} ..')
    read_bytes += os.path.getsize(rf)
    reader = go.rust_api.io.TofPacketReader(rf, filter=go.rust_api.io.PacketType.TofEvent)
    
    for pack in reader:
      ev = go.rust_api.events.TofEvent()
      ev.from_tofpacket(pack)
      for wf in ev.waveforms:
          #print(wf)
          if wf.rb_id == 1:
              finish = True
              break
      if finish:
          break
      if len(ev.waveforms) > 0:
          wf = ev.waveforms[0]
          if len(wf.adc) != 0:
              avg_wf += wf.adc
              nwfs += 1
      npacks += 1  
    break

In [ ]:
wf.calibrate(rbcali)
wf.apply_spike_filter()

In [ ]:
%matplotlib inline

p.plot(wf.times, wf.voltages)

In [ ]:
npacks, nwfs, read_bytes/1e9

In [ ]:
go.rust_api.io.TofPacketReader?

In [ ]:
%matplotlib inline


In [ ]:
output_plots = '/data0/gaps/nevis/test-data/plots'
for rbid in calis:
    fig = p.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
    ax = fig.gca()
    # offsets
    cal = calis[rbid]
    #print (cal.v_offsets)
    #cal.offsets
    for k in range(0,9):
        h = d.factory.hist1d(cal.v_offsets[k], 40)
        h.line(color='k')
    break
    

In [ ]:
%matplotlib inline
p.show()

In [ ]:
data = go.cxx_api.get_tofpackets(str(run0))
n = 0
for k in data:
    print (k)
    n += 1
    if n == 100:
        break

In [ ]:
go.cxx_api.PAMoniData.from_bytestream(data[1].payload,0)

In [ ]:
ev = go.cxx_api.TofEvent.from_bytestream(data[103].payload, 0)

In [ ]:
ev.mt_event

In [ ]:
for rbev in ev.rbevents:
    rbev.plot(calib=calis[f'{rbev.header.rb_id:02}'])
#ev.rbevents[0].plot(calib=calis['31'])

In [ ]:
events = []
for k in go.cxx_api.get_tofpackets(str(run_flight0)):
    if k.packet_type == go.cxx_api.PacketType.TofEvent:
        events.append(go.cxx_api.TofEvent.from_tofpacket(k))
#for k in go.cxx_api.get_tofpackets(str(run_flight1)):
#    if k.packet_type == go.cxx_api.PacketType.TofEvent:
#        events.append(go.cxx_api.TofEvent.from_tofpacket(k))
    

In [ ]:
print (len(events))
for k in events:
    print (k)

In [ ]:
%matplotlib inline

#fig = p.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
#ax  = fig.gca()
nrbs = [len(k.rbevents) for k in events]
print (set(nrbs))

In [ ]:
cb.logger?

In [ ]:
from charmingbeauty.colors import get_color_palette